In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [518]:
from jellyfish.core import Client
from jellyfish.history_loader import load_candles_history
from jellyfish import utils
from datetime import datetime, timedelta

utils.disable_warnings()

pair = 'btcusdt'
mid_dt = datetime(year=2021, month=11, day=25)
size = 5000 * 4

frame = load_candles_history(Client(), pair, end_dt=mid_dt, interval='1m', candles_num=size)

  0%|          | 0/20 [00:00<?, ?it/s]

In [538]:
from jellyfish import indicator
import numpy as np
import pandas as pd

def preprocess(frame: pd.DataFrame):
    returns = frame.Close / frame.Open
    hurst = indicator.hurst(returns, kind=indicator.HURST_CHANGE, simplified=False)

    th = hurst[100:].mean()
    th_eps = hurst[100:].std() * 1e-3
    # th_eps = hurst[100:].std()

    frame['Persistent'] = (hurst > th + th_eps).astype(np.int32)
    frame['Reversal'] = (hurst < th - th_eps).astype(np.int32)
    frame['RandomWalk'] = (np.abs(hurst - th) < th_eps).astype(np.int32)
    frame['i_Hurst'] = hurst
    return frame

frame = preprocess(frame)

  0%|          | 0/19921 [00:00<?, ?it/s]

In [539]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG
from jellyfish.core import SmaCross, Backtest

condition = lambda ohlc: ohlc.Persistent.max() + ohlc.Reversal.max() > 1
frame1 = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')

  0%|          | 0/20020 [00:00<?, ?it/s]

In [543]:
from backtesting.lib import crossover


class MyStrategy(SmaCross):
    rsi_period = 10

    def init(self):
        self.I(indicator.sma, self.data.Close, self.n2)
        self.I(indicator.trima, self.data.Close, self.n2)
        self.I(indicator.zlema, self.data.Close, self.n2)
        self.I(indicator.tema, self.data.Close, self.n2)
        self.I(indicator.vwma, self.data.Close, self.data.Volume, self.n2)

        ha = indicator.heiken_ashi(self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.ha_open = self.I(lambda x: x, ha[0])
        self.ha_close = self.I(lambda x: x, ha[-1])


    def next(self):
        if self.ha_open <= self.ha_close and not self.position.is_long:
            self.position.close()
            self.buy()
        elif self.ha_open >= self.ha_close and not self.position.is_short:
            self.position.close()
            self.sell()
        # if self.vol < 0.07:
        #     self.position.close()
        # else:
        #     super(MyStrategy, self).next()
        # if crossover(self.sma1, self.sma2):
        #     self.position.close()
        #     self.buy()
        # elif crossover(self.sma2, self.sma1):
        #     self.position.close()
        #     self.sell()

bt = Backtest(frame1, MyStrategy, trade_on_close=True)
stats = bt.run(
    n1=8,
    n2=12,
    rsi_period=20
)
bt.plot(open_browser=True)
stats

Start                     2021-11-11 07:14:00
End                       2021-11-25 02:00:00
Duration                     13 days 18:46:00
Exposure Time [%]                   99.844237
Equity Final [$]                768554.153624
Equity Peak [$]                     1000000.0
Return [%]                         -23.144585
Buy & Hold Return [%]              -11.687623
Return (Ann.) [%]                  -99.541287
Volatility (Ann.) [%]                0.319934
Sharpe Ratio                       -11.409967
Calmar Ratio                       -29.375764
Max. Drawdown [%]                  -23.144585
Avg. Drawdown [%]                  -23.144585
Max. Drawdown Duration       13 days 18:34:00
Avg. Drawdown Duration       13 days 18:34:00
# Trades                                  307
Win Rate [%]                        29.967427
Best Trade [%]                       2.878555
Worst Trade [%]                     -3.442241
Avg. Trade [%]                      -0.090719
Max. Trade Duration           0 da

In [521]:
from ta.others import